# Apriori algorithm

Implémentation de l'algorithme *Apriori* sur notre jeu de données. Les métriques calculées sont détaillées ci-dessous, avec des explications adaptées à notre cas d'utilisation et inspirées du tutoriel suivant : https://towardsdatascience.com/apriori-algorithm-for-association-rule-learning-how-to-find-clear-links-between-transactions-bf7ebc22cf0a

## Apprentissage de règles d'association

*Apriori* fait partie des algorithmes d'apprentissage de règles d'association (*association rule learning*), qui appartiennent à la branche non supervisée de l'apprentissage automatique. L'objectif de ces algorithmes est de découvrir des relations intéressantes entre des variables dans de grandes bases de données.

## Métriques

### Support
La première étape consiste à trouver les GHM fréquents. Le calcul du support est basé sur la fréquence : \
\
$Support(GHM_i) = \frac{\text{Nb d'hospitalisations pour le } GHM_i}{\text{Nb total d'hospitalisations}}$

### Confidence
La confiance nous indique à quel point nous pouvons être sûrs (d'après nos données) qu'un GHM se trouvera dans le parcours de soins d'un patient, étant donné qu'un autre GHM a été observé.\
\
$Confidence(GHM_i\longrightarrow GHM_j) = \frac{Support(GHM_i \cup GHM_j)}{Support(GHM_i)}$

### Lift

Lift est une mesure qui nous indique si la probabilité d'apparition d'un GHM augmente ou diminue compte tenu de la présence d'un autre GHM dans le parcours de soin.\
\
$Lift(GHM_i \longrightarrow GHM_j) = \frac{\mathbb{P}(GHM_i \cup GHM_j)}{Support(GHM_i) \times Support(GHM_j)}$ \
\
On peut simplifier cette formule en remplaçant
$\frac{\mathbb{P}(GHM_i \cup GHM_j)}{Support(GHM_i)}$
par $Confidence(GHM_i\longrightarrow GHM_j)$. Ansi, on obtient : \
\
$Lift(GHM_i \longrightarrow GHM_j) = \frac{Confidence(GHM_i\longrightarrow GHM_j)}{Support(GHM_j)}$

+ $Lift>1$ : les deux GHM sont plus susceptibles d'être présents dans un même parcours de soins
+ $Lift<1$ : les deux GHM sont plus susceptibles d'être présents dans des parcours de soins différents
+ $Lift=1$ : il n'y a pas d'association entre les deux GHM

## Implémentation

In [1]:
!pip install apyori

In [10]:
from apyori import apriori
import pandas as pd
import pickle

In [3]:
def generate_output(db):
    association_rules = apriori(db,
                            min_support=0.02,
                            min_confidence=0.05,
                            #min_lift=1,
                            max_length=None)
    
    association_results = list(association_rules)
    
    listRules = [list(association_results[i][0]) for i in range(0,len(association_results))]
    listSupport = [association_results[i].support for i in range(0,len(association_results))]
    listConfidence = [association_results[i].ordered_statistics[0].confidence for i in range(0,len(association_results))]
    listLift = [association_results[i].ordered_statistics[0].lift for i in range(0,len(association_results))]
    
    return pd.DataFrame({"Motif":listRules,
                         "Support":listSupport,
                         "Confidence":listConfidence,
                         "Lift":listLift}).sort_values(by="Support", ascending=False).reset_index(drop=True)

In [4]:
%%capture capt
p_soins=pd.read_csv("../data/parcours_soins.csv")
patients=pd.read_csv("../data/profil_patient.csv")

In [5]:
%%capture capt
p_soins=p_soins.rename(columns={"BEN_NIR_IDT": "CODE_PATIENT"})
p_soins=p_soins.set_index("CODE_PATIENT")
patients=patients.set_index("CODE_PATIENT")
p_soins["cluster"]=patients["cluster"]

In [6]:
with open('../data/parcours_soins.pickle', 'rb') as handle:
    p_soin=pickle.load(handle)
    
with open('../data/parcours_soins_dp.pickle', 'rb') as handle:
    p_soin_dp=pickle.load(handle)

In [7]:
corpus = []
for i in range(18):
    corpus += p_soin[f"Cluster {i}"]

In [8]:
len(corpus)

10051

### Sortie pour l'ensemble des parcours de soins

In [9]:
generate_output(corpus)

,Motif,Support,Confidence,Lift
0,[05M092],0.182867,0.182867,1.000000
1,[05M093],0.162272,0.162272,1.000000
2,[02C05J],0.138295,0.138295,1.000000
3,[05K101],0.098000,0.098000,1.000000
4,[04M053],0.085066,0.085066,1.000000
5,[05M091],0.079893,0.079893,1.000000
6,[05M09T],0.078599,0.078599,1.000000
7,[23M20Z],0.066063,0.066063,1.000000
8,[23M20T],0.058104,0.058104,1.000000
9,[06K04J],0.055218,0.055218,1.000000


In [11]:
corpus_dp = []
for i in range(18):
    corpus_dp += p_soin_dp[f"Cluster {i}"]

In [12]:
len(corpus_dp)

10051

In [13]:
generate_output(corpus_dp)

,Motif,Support,Confidence,Lift
0,[I500],0.232415,0.232415,1.000000
1,[I501],0.214705,0.214705,1.000000
2,[H251],0.069048,0.069048,1.000000
3,"[I501, I500]",0.068550,0.068550,1.000000
4,[I509],0.067456,0.067456,1.000000
5,[Z098],0.061785,0.061785,1.000000
6,[I48],0.055517,0.055517,1.000000
7,[J960],0.050244,0.050244,1.000000
8,[Z515],0.050045,0.050045,1.000000
9,"[I509, I500]",0.025968,0.111729,1.656331
